In [1]:
# 필요한 모듈 임포트
import os
from dotenv import load_dotenv
import httpx

class CustomHTTPClient(httpx.Client):
    def __init__(self, *args, **kwargs):
        kwargs.pop("proxies", None) # Remove the 'proxies' argument if present
        super().__init__(*args, **kwargs)

from openai import OpenAI

#환경변수 취득
load_dotenv()
#OpenAI API 클라이언트 생성
client = OpenAI(api_key=os.environ['API_KEY'], http_client=CustomHTTPClient())

# 모델명
model_name = "gpt-3.5-turbo-0125"

In [2]:
import json

def get_search_result(keyword, when, unit="d"):
    result = {
        "result": [
            {"title": "1900년(광무 4년) 경인선의 서울 도심 구간 개통과 함께 남대문정거장으로 개업"},
            {"title": "1923년 경성역(京城驛)으로 이름이 바뀐 뒤 광복 후인 1947년에 현재의 서울역으로 이름이 바뀌었다. "},
            {"title": "2017년 11월 28일을 기하여 더 이상 수도권 전철 경의·중앙선의 여객 취급은 하지 않게 되었고, 대신 인천공항에서 들어오는 2018 평창 동계올림픽 수송용 KTX 임시편성을 취급하게 되었다. 경의·중앙선은 구 서울역 쪽으로 이설해 취급한다."},
        ]
    }

    return json.dumps(result)

In [3]:
from openai.types.chat import ChatCompletionToolParam

# 툴 정의
tools = [
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "지정한 키워드의 검색 결과를 취득한다",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "키워드"
                    },
                    "when": {
                        "type": "number",
                        "description": "날짜/시간의 범주"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"]
            },
        },
    })
]

In [27]:
question = "서울역의 이벤트에 관해, 최근 1개월 이내의 검색결과를 알려줘"

response = client.chat.completions.create(
    model = model_name,
    messages = [
        {"role": "system", "content": "당신은 JSON형식으로 출력하도록 설계된 도우미입니다."},
        {"role": "user", "content": question},

    ],
    tools=tools,
    tool_choice="auto",
    response_format={"type": "json_object"}
)

In [5]:
response

ChatCompletion(id='chatcmpl-BIX7duH6oYFekrCZA7QmGEwu8Fkn0', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_A6n1vbrtJuZLB1SnQI6Mst7o', function=Function(arguments='{"keyword":"서울역 이벤트","when":1,"unit":"m"}', name='get_search_result'), type='function')]))], created=1743756925, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=32, prompt_tokens=154, total_tokens=186, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [15]:
response.choices[0].message.tool_calls[0].function.arguments

'{"keyword":"서울역","when":1,"unit":"m"}'

In [28]:
# 모델이 툴을 부르는지 판단
if response.choices[0].message.tool_calls is not None:
    # 언어 모델 응답을 취득
    tool = response.choices[0].message.tool_calls[0]

    # 함수명 취득
    function_name = tool.function.name
    print(f"함수명: {function_name}")
    # 인수의 취득
    arguments = json.loads(tool.function.arguments)
    print(f"인수: {arguments}")

    #함수명으로 실행하는 함수를 판단
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword = arguments.get("keyword"),
            when = arguments.get("when"),
            unit = arguments.get("unit")
        )
        print(f"함수의 실행 결과: {function_response}")

        #함수의 실행결과를 messages에 추가해서 언어모델을 호출
        response2 = client.chat.completions.create(
            model = model_name,
            messages = [
                {"role": "user", "content": question},
                response.choices[0].message,
                {
                    "tool_call_id": tool.id,
                    "role": "tool",
                    "content": function_response,
                }
            ]
        )

        #언어 모델로부터 응답을 출력
        print(response2.choices[0].message.content.strip())
else:
    #함수를 부르지 않는다면, 단순한 언어모델로부터 응답을 출력
    print(response.choices[0].message.content.strip())        

함수명: get_search_result
인수: {'keyword': '서울역', 'when': 1, 'unit': 'm'}
함수의 실행 결과: {"result": [{"title": "1900\ub144(\uad11\ubb34 4\ub144) \uacbd\uc778\uc120\uc758 \uc11c\uc6b8 \ub3c4\uc2ec \uad6c\uac04 \uac1c\ud1b5\uacfc \ud568\uaed8 \ub0a8\ub300\ubb38\uc815\uac70\uc7a5\uc73c\ub85c \uac1c\uc5c5"}, {"title": "1923\ub144 \uacbd\uc131\uc5ed(\u4eac\u57ce\u9a5b)\uc73c\ub85c \uc774\ub984\uc774 \ubc14\ub010 \ub4a4 \uad11\ubcf5 \ud6c4\uc778 1947\ub144\uc5d0 \ud604\uc7ac\uc758 \uc11c\uc6b8\uc5ed\uc73c\ub85c \uc774\ub984\uc774 \ubc14\ub00c\uc5c8\ub2e4. "}, {"title": "2017\ub144 11\uc6d4 28\uc77c\uc744 \uae30\ud558\uc5ec \ub354 \uc774\uc0c1 \uc218\ub3c4\uad8c \uc804\ucca0 \uacbd\uc758\u00b7\uc911\uc559\uc120\uc758 \uc5ec\uac1d \ucde8\uae09\uc740 \ud558\uc9c0 \uc54a\uac8c \ub418\uc5c8\uace0, \ub300\uc2e0 \uc778\ucc9c\uacf5\ud56d\uc5d0\uc11c \ub4e4\uc5b4\uc624\ub294 2018 \ud3c9\ucc3d \ub3d9\uacc4\uc62c\ub9bc\ud53d \uc218\uc1a1\uc6a9 KTX \uc784\uc2dc\ud3b8\uc131\uc744 \ucde8\uae09\ud558\uac8c \ub418\